# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [30]:
import numpy as np
import glob
import sys
import os.path
import fnmatch
import pandas as pd

data_dir = '/hamidb/applications/udacity-carnd/repo/data'
headers = ["center", "left", "right", "steer_angle",
           "throttle", "break", "speed"]
logs = []
for setpath in sorted(glob.glob(data_dir+"/*")):
    setname = os.path.basename(setpath)
    for contents in os.listdir(setpath):
        if fnmatch.fnmatch(contents,'*.csv'):
            log_file = os.path.join(setpath, contents)
            
            log = pd.read_csv(log_file, header=None, names=headers, skiprows=1)
            logs.append(log)
    
logs = pd.concat(logs, axis=0, ignore_index=True)

print(logs.shape)



(8036, 7)


In [31]:
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from PIL import Image

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

dir_prefix = "/hamidb/applications/udacity-carnd/repo/data/set1"

features = []
labels = []

target_size = (64, 64)

def batch_generator(batch_size, data):
    
    
for item in zip(logs['center'], logs['steer_angle']):
    img = image.load_img(os.path.join(dir_prefix, item[0]), target_size=(128, 128))
    x = image.img_to_array(img)
    #x = x[60:,:,:]
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features.append(x[0])
    labels.append(item[1])

features = np.array(features)
labels = np.array(labels)
print(features.shape)
print(labels.shape)

# Get randomized datasets for training and test
X_train, X_test, y_train, y_test = train_test_split(
    features,
    labels,
    test_size=0.10,
    random_state=43)

# Get randomized datasets for training and validation
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train,
    y_train,
    test_size=0.10,
    random_state=43)

# Print out shapes of new arrays
train_size = X_train.shape[0]
test_size = X_test.shape[0]
valid_size = X_valid.shape[0]
input_shape = X_train.shape[1:]

print("train size:", train_size)
print("valid size:", valid_size)
print("test size:", test_size)
print("input shape:", input_shape)


(8036, 128, 128, 3)
(8036,)
train size: 6508
valid size: 724
test size: 804
input shape: (128, 128, 3)


In [8]:
import pickle

pickle_file = 'simulator.pickle'

with open(pickle_file, 'wb') as pfile:
    pickle.dump({'train_dataset': X_train,
                 'train_labels': y_train,
                 'valid_dataset': X_valid,
                 'valid_labels': y_valid,
                 'test_dataset': X_test,
                 'test_labels': y_test,},
                pfile, pickle.HIGHEST_PROTOCOL)
        

In [29]:
from keras.applications.vgg16 import VGG16 as vgg16
from keras.layers import Dense, Input, AveragePooling2D
from keras.layers import Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam


base_model = vgg16(weights='imagenet', input_tensor=Input((64, 64, 3)), include_top=False)

# freeze all convolutional layers
for layer in base_model.layers:
    layer.trainable = False
    
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(1)(x)

#x = BatchNormalization()(x)
#x = Dropout(0.5)(x)
#x = Flatten()(x)
#x = Dense(4096, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = Dense(2048, activation="relu")(x)
# x = Dense(2048, activation="relu")(x)
# x = Dense(1024, activation="relu")(x)
# x = Dense(1, activation="linear")(x)
# this is the model we will train
model = Model(input=base_model.input, output=x)

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 128, 128, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 128, 128, 64)  0           input_7[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 128, 128, 64)  0           block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 64, 64, 64)    0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [26]:
# TODO: Compile and train the model here.
batch_size = 64
nb_epoch = 9

model.compile(loss='mse',
              optimizer=Adam(),
              metrics=['accuracy', 'mean_absolute_error'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_valid, y_valid))
#validation_split=0.15


Train on 6508 samples, validate on 724 samples
Epoch 1/9
6508/6508 [==============================] - 3364s - loss: 0.0332 - acc: 0.5353 - mean_absolute_error: 0.0935 - val_loss: 0.0184 - val_acc: 0.5387 - val_mean_absolute_error: 0.0787
Epoch 2/9
6508/6508 [==============================] - 3364s - loss: 0.0168 - acc: 0.5403 - mean_absolute_error: 0.0758 - val_loss: 0.0180 - val_acc: 0.5387 - val_mean_absolute_error: 0.0778
Epoch 3/9
6508/6508 [==============================] - 3363s - loss: 0.0168 - acc: 0.5403 - mean_absolute_error: 0.0761 - val_loss: 0.0182 - val_acc: 0.5387 - val_mean_absolute_error: 0.0817
Epoch 4/9
6508/6508 [==============================] - 3358s - loss: 0.0167 - acc: 0.5403 - mean_absolute_error: 0.0756 - val_loss: 0.0180 - val_acc: 0.5387 - val_mean_absolute_error: 0.0748
Epoch 5/9
6508/6508 [==============================] - 3366s - loss: 0.0167 - acc: 0.5403 - mean_absolute_error: 0.0741 - val_loss: 416586848253583622144.0000 - val_acc: 0.0000e+00 - val_me

In [11]:
score = model.evaluate(X_test[0:100], y_test[0:100], verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

100/100 [==============================] - 4s     
Test score: 0.00970136054792
Test accuracy: 0.52


In [27]:
import json
import h5py

with open('model.json', 'w') as f:
    json.dump(model.to_json(), f)

model.save_weights('model.h5')

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

**Validation Accuracy**: (fill in here)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [ ]:
with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

model.evaluate(X_test, Y_test)

**Test Accuracy:** (fill in here)

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.